<a href="https://colab.research.google.com/github/TheMihirNaik/google-search-console-api-using-python/blob/main/GSC_API_with_Python_First_Video_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required python packages
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2

In [ ]:
# Import required packages
import httplib2
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build

In [ ]:
# Google Cloud Project Credentials
CLIENT_ID = '544906092305-se649hqg0c9jvhn80o9fdcui6horn77r.apps.googleusercontent.com'
CLIENT_SECRET = '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'

# Define oauth scopes, the following is read/write access
OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters'

# The following is a read only access
#OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly' 

# Redirect URI for Google Colab specifically
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

# Build URL that generates Authorization Code
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()

#print authorization URL
print ('Go to the following link in your browser: ' + authorize_url)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=544906092305-se649hqg0c9jvhn80o9fdcui6horn77r.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters&access_type=offline&response_type=code


In [ ]:
# Recieve Authorization Code
auth_code = input('Enter your Authorization Code here : ')


# gets credentials to build service
credentials = flow.step2_exchange(auth_code)
print('The credentials are generated')

Enter your Authorization Code here : 4/1AWtgzh7daLDsgH5ZU9luJf-kNbs2u_Hda5CTGkK_MDk7NH07T4bEI26DJso
The credentials are generated


In [ ]:
# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()

#authorize credentials
creds = credentials.authorize(http)

#build a service to access various features of GSC API
webmasters_service = build('searchconsole', 'v1', http=creds)

How to get a list of Sites in GSC account?

In [ ]:
# Retrieve list of properties in account
site_list = webmasters_service.sites().list().execute()

# Filter for verified websites
verified_sites_urls = [s['siteUrl'] for s in site_list['siteEntry']
                    if s['permissionLevel'] != 'siteUnverifiedUser'
                        and s['siteUrl'][:4] == 'http']

print(verified_sites_urls)

['https://www.tridentpbi.in/', 'https://moneymonkdigital.com/']


How to get sitemap data?

In [ ]:
# what website do you want to get the data for?
url = 'https://www.tridentpbi.in/'

#get sitemap data
sitemap_data = webmasters_service.sitemaps().list(siteUrl=url).execute()

print(sitemap_data)

{'sitemap': [{'path': 'https://www.tridentpbi.in/sitemap_index.xml', 'lastSubmitted': '2023-03-07T03:44:02.448Z', 'isPending': False, 'isSitemapsIndex': True, 'lastDownloaded': '2023-03-08T21:39:08.727Z', 'warnings': '5', 'errors': '0', 'contents': [{'type': 'web', 'submitted': '26', 'indexed': '0'}, {'type': 'image', 'submitted': '194', 'indexed': '0'}]}]}


In [ ]:
# locate the exact sitemap URL
sitemap_loc = sitemap_data['sitemap'][0]['path']

#print the sitemap url
print(sitemap_loc)

https://www.tridentpbi.in/sitemap_index.xml


*How* to do URL inspection?

In [ ]:
url_to_inspect = 'https://www.tridentpbi.in/products/corrugated-box-cardboard-cartons/5-ply/'


request = {
  'inspectionUrl': url_to_inspect,
  'siteUrl': url
}
 
response = webmasters_service.urlInspection().index().inspect(body=request).execute()
inspectionResult = response['inspectionResult']

#show the result on screen
inspectionResult

{'inspectionResultLink': 'https://search.google.com/search-console/inspect?resource_id=https://www.tridentpbi.in/&id=956EZpTxPfhiViJ93m2WWw&utm_medium=link&utm_source=api',
 'indexStatusResult': {'verdict': 'PASS',
  'coverageState': 'Submitted and indexed',
  'robotsTxtState': 'ALLOWED',
  'indexingState': 'INDEXING_ALLOWED',
  'lastCrawlTime': '2023-03-01T23:46:03Z',
  'pageFetchState': 'SUCCESSFUL',
  'googleCanonical': 'https://www.tridentpbi.in/products/corrugated-box-cardboard-cartons/5-ply/',
  'userCanonical': 'https://www.tridentpbi.in/products/corrugated-box-cardboard-cartons/5-ply/',
  'referringUrls': ['https://www.tridentpbi.in/',
   'https://www.tridentpbi.in/products/industries/fruits-vegetables/'],
  'crawledAs': 'MOBILE'},
 'mobileUsabilityResult': {'verdict': 'FAIL',
  'issues': [{'issueType': 'TAP_TARGETS_TOO_CLOSE',
    'message': 'Clickable elements too close together'},
   {'issueType': 'SIZE_CONTENT_TO_VIEWPORT',
    'message': 'Content wider than screen'}]}}

How to get Search Analytics Data?

In [ ]:
#get google search console data

# what website do you want to get the data for?
url = 'https://www.tridentpbi.in/'

#build a request body
request_body = {
    "startDate" : '2022-01-01', 
    "endDate" : '2022-12-31', 
    "aggregationType" : "AUTO", 
    "dataState" : "ALL", 
    "dimensions" : [ "DATE", "QUERY", "PAGE", "COUNTRY", "DEVICE"], 
    "type": "WEB", 
    "rowLimit" : 25000}

#get the response using request body
response_data = webmasters_service.searchanalytics().query(siteUrl=url, body=request_body).execute()

In [ ]:
response_data['rows'][0]

{'keys': ['2022-07-25',
  'trident paper box industries',
  'https://www.tridentpbi.in/',
  'ind',
  'DESKTOP'],
 'clicks': 7,
 'impressions': 9,
 'ctr': 0.7777777777777778,
 'position': 1}

Unlimited Google Search Console Data

In [ ]:
all_responses = []
startRow = 0
totalRow = 0

if totalRow == 0:
  request = {"startDate": '2022-01-01', "endDate": '2023-02-28', "aggregationType": "BY_PAGE", "dataState": "ALL", "dimensions": [ "DATE", "QUERY", "PAGE", "COUNTRY", "DEVICE"], "type": "WEB", "rowLimit" : 25000, "startRow" : startRow}
  response = webmasters_service.searchanalytics().query(siteUrl=url, body=request).execute()
  totalRow = totalRow + len(response['rows'])
  startRow = startRow + 25000
  print(request)
  for each in response['rows']:
    all_responses.append(each)
  
  while totalRow%25000 == 0:
    request = {"startDate": '2022-01-01', "endDate": '2023-02-28', "aggregationType": "BY_PAGE", "dataState": "ALL", "dimensions": [ "DATE", "QUERY", "PAGE", "COUNTRY", "DEVICE"], "type": "WEB", "rowLimit" : 25000, "startRow" : startRow}
    response = webmasters_service.searchanalytics().query(siteUrl=url, body=request).execute()
    totalRow = totalRow + len(response['rows'])
    startRow = startRow + 25000
    print(request)
    for each in response['rows']:
      all_responses.append(each)

{'startDate': '2022-01-01', 'endDate': '2023-02-28', 'aggregationType': 'BY_PAGE', 'dataState': 'ALL', 'dimensions': ['DATE', 'QUERY', 'PAGE', 'COUNTRY', 'DEVICE'], 'type': 'WEB', 'rowLimit': 25000, 'startRow': 0}
{'startDate': '2022-01-01', 'endDate': '2023-02-28', 'aggregationType': 'BY_PAGE', 'dataState': 'ALL', 'dimensions': ['DATE', 'QUERY', 'PAGE', 'COUNTRY', 'DEVICE'], 'type': 'WEB', 'rowLimit': 25000, 'startRow': 25000}
{'startDate': '2022-01-01', 'endDate': '2023-02-28', 'aggregationType': 'BY_PAGE', 'dataState': 'ALL', 'dimensions': ['DATE', 'QUERY', 'PAGE', 'COUNTRY', 'DEVICE'], 'type': 'WEB', 'rowLimit': 25000, 'startRow': 50000}
{'startDate': '2022-01-01', 'endDate': '2023-02-28', 'aggregationType': 'BY_PAGE', 'dataState': 'ALL', 'dimensions': ['DATE', 'QUERY', 'PAGE', 'COUNTRY', 'DEVICE'], 'type': 'WEB', 'rowLimit': 25000, 'startRow': 75000}
{'startDate': '2022-01-01', 'endDate': '2023-02-28', 'aggregationType': 'BY_PAGE', 'dataState': 'ALL', 'dimensions': ['DATE', 'QUERY

In [ ]:
len(all_responses)

364513

In [ ]:
data = []

for each in all_responses:
  temp = []
  temp.append(each['clicks'])
  temp.append(round(each['ctr']*100, 2))
  temp.append(each['impressions'])
  temp.append(each['keys'][0])
  temp.append(each['keys'][1])
  temp.append(each['keys'][2])
  temp.append(each['keys'][3])
  temp.append(each['keys'][4])
  temp.append(each['position'])
  temp.append(round(each['position'], 0))
  data.append(temp)
  

In [ ]:
import pandas as pd
df = pd.DataFrame(data, columns =['clicks', 'ctr', 'impressions', 'date','query', 'page', 'country', 'device', 'position', 'round_position'])

In [ ]:
df

,clicks,ctr,impressions,date,query,page,country,device,position,round_position
0,7,77.78,9,2022-07-25,trident paper box industries,https://www.tridentpbi.in/,ind,DESKTOP,1.0,1.0
1,6,85.71,7,2022-01-20,trident paper box industries,https://www.tridentpbi.in/,ind,DESKTOP,1.0,1.0
2,6,100.00,6,2022-02-25,trident paper box industries,https://www.tridentpbi.in/,ind,DESKTOP,1.0,1.0
3,6,85.71,7,2022-09-14,trident paper box industries,https://www.tridentpbi.in/,ind,DESKTOP,1.0,1.0
4,6,85.71,7,2023-02-22,trident paper box industries,https://www.tridentpbi.in/,ind,DESKTOP,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
364508,0,0.00,2,2023-02-28,white corrugated boxes,https://www.tridentpbi.in/products/corrugated-...,ind,DESKTOP,14.5,14.0
364509,0,0.00,1,2023-02-28,white corrugated boxes wholesale,https://www.tridentpbi.in/products/corrugated-...,can,DESKTOP,26.0,26.0
364510,0,0.00,1,2023-02-28,white corrugated boxes wholesale,https://www.tridentpbi.in/products/corrugated-...,mys,DESKTOP,27.0,27.0
364511,0,0.00,1,2023-02-28,white corrugated boxes wholesale,https://www.tridentpbi.in/products/corrugated-...,usa,DESKTOP,79.0,79.0
